# Issue Commands to Change Settings

In [26]:
import context

from gopro_helper import gopro
from gopro_helper import api
from gopro_helper import commands

# Camera Current Settings

In [27]:
k = gopro.GoProStatus(interval=1)

In [25]:
k.stop()

# Commands

In [39]:
commands.set_mode_photo()

In [38]:
commands.set_mode_video()

In [53]:
commands.shutter_capture()

In [52]:
commands.shutter_stop()

In [48]:
api.url_browse

'http://10.5.5.9:8080/videos/DCIM'

In [19]:
for item in api.api_details['camera_mode_map']:
    print('{:20s}: {}'.format(item['wsdk_mode_group_key'], item['mode_value']))
    print('{:20s}: {}'.format(item['wsdk_mode_key'], item['sub_mode_value']))
    print()

video               : 0
video               : 0

multishot           : 0
video_time_lapse    : 1

video               : 0
video_plus_photo    : 2

video               : 0
looping             : 3

photo               : 1
photo               : 0

photo               : 1
photo               : 1

photo               : 1
night               : 2

photo               : 2
burst               : 0

multishot           : 2
photo_time_lapse    : 1

multishot           : 2
photo_night_lapse   : 2

